In [1]:

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import time

# BERT files

#os.listdir("../input/pretrained-bert-including-scripts/master/bert-master")
#os.listdir('gdrive/My Drive/kaggle/jigsaw/data/bert_tensorflow/bert-master')

#sys.path.insert(0, '../input/pretrained-bert-including-scripts/master/bert-master')
sys.path.insert(0, '/content/gdrive/My Drive/kaggle/jigsaw/data/bert_tensorflow/bert-master')


import tensorflow_hub as hub

from run_classifier import *
import modeling
import optimization
import tokenization


#import run_classifier
import run_classifier_with_tfhub
#import tokenization

#import logging

W0609 01:21:59.453497 140435720193920 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
import pprint
import json
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  
  
  
BUCKET = 'jigsawbert' #@param {type:"string"}
TASK = 'CoLA_weights'
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

TPU address is grpc://10.1.116.138:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3312154910133415446),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9475530081492378002),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15409016772561070563),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16799074215003827883),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 6212033213174644562),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6218435986248858940),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 18144644100255875731),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 8193776339599055240),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1302753029116

In [0]:
# import data

train=pd.read_csv('gdrive/My Drive/kaggle/jigsaw/data/train.csv') #, nrows = 50000)
test=pd.read_csv('gdrive/My Drive/kaggle/jigsaw/data/test.csv')
train=train.fillna(0)

# remove new lines etc. -- TODO
train['comment_text'] = train['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True).replace(r'\t',  ' ', regex=True)
test['comment_text'] = test['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True).replace(r'\t',  ' ', regex=True)

# force train into cola format, test is fine as it is  -- I don't know what is it
train['dummy_1'] = 'meh'
train['dummy_2'] = '*'


In [0]:
TOXICITY_COLUMN = 'target'
identity_columns = ['asian', 'atheist',
       'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender',
       'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']
    
subgroup_bool_train = train[identity_columns].fillna(0)>=0.5
toxic_bool_train = train[TOXICITY_COLUMN].fillna(0)>=0.5
#subgroup_negative_mask = subgroup_bool_train.values.sum(axis=1).astype(bool) & ~toxic_bool_train

bpsn = ((subgroup_bool_train.values.sum(axis=1).astype(bool) & ~toxic_bool_train)).astype(int)   +  \
 ((~subgroup_bool_train.values.sum(axis=1).astype(bool)  & toxic_bool_train)).astype(int) 

# Overall
weights = np.ones((len(train),))
# Subgroup negative
weights += bpsn #subgroup_negative_mask
loss_weight = 1.0 / weights.mean()

train['weights'] = weights

train['target'] = np.where(train['target']>=0.5,1,0)

In [6]:
val = 100000
trainrows = len(train)
dev = train.iloc[trainrows-val:]
devcopy = dev.copy()
train = train.iloc[0:trainrows-val]

train = train[['dummy_1','target','dummy_2','comment_text','weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
dev = dev[['dummy_1','target','dummy_2','comment_text','weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
print(dev.shape, train.shape, devcopy.shape)

# export as tab seperated
train.to_csv(  'train.tsv', sep='\t', index=False, header=False)
test.to_csv( 'test.tsv', sep='\t', index=False, header=True)
dev.to_csv('dev.tsv', sep='\t', index=False, header=False)


(100000, 10) (1704874, 10) (100000, 48)


In [7]:
train.target.value_counts(normalize=True), dev.target.value_counts(normalize=True)

(0    0.92022
 1    0.07978
 Name: target, dtype: float64, 0    0.9168
 1    0.0832
 Name: target, dtype: float64)

In [8]:
!gsutil cp 'train.tsv' 'gs://jigsawbert/bert-tfhub/models/CoLA_weights/'
!gsutil cp 'test.tsv' 'gs://jigsawbert/bert-tfhub/models/CoLA_weights/'
!gsutil cp 'dev.tsv' 'gs://jigsawbert/bert-tfhub/models/CoLA_weights/'

Copying file://train.tsv [Content-Type=text/tab-separated-values]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/546.5 MiB.                                    
Copying file://test.tsv [Content-Type=text/tab-separated-values]...
-
Operation completed over 1 objects/28.7 MiB.                                     
Copying file://dev.

In [0]:
#Parameters

task_name = 'cola'

BERT_BASE = 'gs://jigsawbert/uncased_l-12_h-768_a-12'

LABEL_COLUMNS = ['target','weights', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat' ]

bert_config_file = 'gdrive/My Drive/kaggle/jigsaw/data/uncased_L-12_H-768_A-12/bert_config.json'
vocab_file = 'gdrive/My Drive/kaggle/jigsaw/data/uncased_L-12_H-768_A-12/vocab.txt'
init_checkpoint = 'gdrive/My Drive/kaggle/jigsaw/data/uncased_L-12_H-768_A-12/bert_model.ckpt'
data_dir =  OUTPUT_DIR


do_lower_case = True
max_seq_length = 222 #72
do_train = True
do_eval = True
do_predict = True
train_batch_size = 32 #256 
eval_batch_size = 32
predict_batch_size = 32
learning_rate = 2e-5 #4e-5   
num_train_epochs = 2.0
warmup_proportion = 0.05  #0.05 #0.1  
use_tpu = True 
save_checkpoints_steps = 30000 #99999999 # <----- don't want to save any checkpoints
iterations_per_loop = 1000
num_tpu_cores = 8
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

In [0]:

def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    
    probabilities = tf.nn.sigmoid(logits)
  
    per_example_loss1 = tf.nn.weighted_cross_entropy_with_logits(pos_weight=labels[:,1], targets=labels[:,0], logits=logits[:,0])
    per_example_loss2 = tf.nn.sigmoid_cross_entropy_with_logits( labels=labels[:,2:], logits=logits[:,2:])
    per_example_loss = (per_example_loss1 * loss_weight) + tf.reduce_mean(per_example_loss2,axis=1)
    
    loss = tf.reduce_mean(per_example_loss)
    
    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      
      #logging_hook = tf.train.LoggingTensorHook({"total_loss": total_loss}, every_n_iter=10)
      
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
      
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        #accuracy = tf.metrics.accuracy(label_ids, predictions)
        #auc = tf.metrics.auc(label_ids,tf.nn.sigmoid(logits[:,1]))
        
        auc = tf.metrics.auc(label_ids[:,0], tf.nn.sigmoid(logits[:,0]))
        
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_AUC": auc, 
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities})
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
#Fine Tuning

processor = ColaProcessor()
label_list = processor.get_labels()
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [0]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

run_config = tf.contrib.tpu.RunConfig(
  cluster=tpu_cluster_resolver,
  #master=master,
  model_dir=OUTPUT_DIR,  #output_dir,
  save_checkpoints_steps=save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=iterations_per_loop,
      num_shards=num_tpu_cores,
      per_host_input_for_training=is_per_host))

In [0]:
def create_examples(lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      #print(line)
      # Only the test set has a header
      if set_type == "test" and i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      if set_type == "test":
        text_a = tokenization.convert_to_unicode(line[1])
        label = "0"
      else:
        text_a = tokenization.convert_to_unicode(line[3])
        #label = tokenization.convert_to_unicode(line[1])
        label =  tokenization.convert_to_unicode(line[1] + ' ' + ' '.join(line[4:])).split()
        #print(label)
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples
  

In [0]:
def read_tsv( input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines
    


In [17]:

train_examples = create_examples(read_tsv(os.path.join(data_dir, "train.tsv")),"train")

num_train_steps = int(len(train_examples) / train_batch_size * num_train_epochs)
num_warmup_steps = int(num_train_steps * warmup_proportion)

os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

print(num_train_steps, num_warmup_steps)

106554 5327


In [18]:
import gc
gc.collect()

0

In [19]:

model_fn = model_fn_builder(
  num_labels=len(LABEL_COLUMNS),   
  learning_rate=learning_rate,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=use_tpu,
  bert_hub_module_handle=BERT_MODEL_HUB) 


estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size,
      eval_batch_size = eval_batch_size,
      eval_on_tpu = use_tpu,
      predict_batch_size=predict_batch_size)


W0609 01:34:29.230513 140435720193920 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fb918122730>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://jigsawbert/bert-tfhub/models/CoLA_weights', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 30000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.1.116.138:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb964f20b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.1.116.138:8470', '_evaluation_master': 'grpc://10.1.116.138:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8,

I0609 01:34:29.236222 140435720193920 estimator.py:201] Using config: {'_model_dir': 'gs://jigsawbert/bert-tfhub/models/CoLA_weights', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 30000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.1.116.138:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb964f20b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.1.116.138:8470', '_evaluation_master': 'grpc://10.1.116.138:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0609 01:34:29.239873 140435720193920 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""
  
   
  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=[0]*len(LABEL_COLUMNS),
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  #label_id = label_map[example.label]
  labels_ids = []
  #print(ex_index, example.label)
  for label in example.label:
    labels_ids.append(float(label))
  
  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %s)" % (example.label, labels_ids))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=labels_ids,
      is_real_example=True)
  return feature


def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)
    
    
    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f
    
    
    def create_float_feature(values):
      f = tf.train.Feature(float_list = tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    
    features["label_ids"] = create_float_feature(feature.label_id)
    
    #features["label_ids"] = create_int_feature([feature.label_id])
    
    
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()


In [0]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([len(LABEL_COLUMNS)], tf.float32),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [22]:
train_file = os.path.join(OUTPUT_DIR, "train.tf_record")
if not tf.io.gfile.exists(train_file):
  print('no')
  
  
import gc
gc.collect()

no


552

In [23]:
start = time.time()
print("--------------------------------------------------------")
print("Starting TF Record...")
print("--------------------------------------------------------")

#train_file = os.path.join(output_dir, "train.tf_record")
train_file = os.path.join(OUTPUT_DIR, "train.tf_record")

label_list = [0] * len(LABEL_COLUMNS)

if not tf.io.gfile.exists(train_file):
  file_based_convert_examples_to_features(
      train_examples, label_list, max_seq_length, tokenizer, train_file)

end = time.time()  
print("--------------------------------------------------------")
print("TF Record done in", end - start, " seconds")
print("--------------------------------------------------------")
  
start = time.time()
print("--------------------------------------------------------")
print("Starting training...")
print("--------------------------------------------------------")


tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=max_seq_length,
    is_training=True,
    drop_remainder=True)


estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

end = time.time()
print("--------------------------------------------------------")
print("Training complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting TF Record...
--------------------------------------------------------
INFO:tensorflow:Writing example 0 of 1704874


I0609 01:34:38.714938 140435720193920 <ipython-input-20-d5b71b59cc05>:117] Writing example 0 of 1704874


INFO:tensorflow:*** Example ***


I0609 01:34:38.723184 140435720193920 <ipython-input-20-d5b71b59cc05>:91] *** Example ***


INFO:tensorflow:guid: train-0


...

INFO:tensorflow:Loss for final step: 0.028177084.


I0609 04:45:29.037068 140435720193920 estimator.py:359] Loss for final step: 0.028177084.


INFO:tensorflow:training_loop marked as finished


I0609 04:45:29.046032 140435720193920 error_handling.py:93] training_loop marked as finished


--------------------------------------------------------
Training complete in  8730.101613998413  seconds
--------------------------------------------------------


In [24]:
import gc
gc.collect()

1886381

In [25]:
#Export Model 
def serving_input_fn(): 
    label_ids = tf.placeholder(tf.float32, [None,len(LABEL_COLUMNS)], name='label_ids')
    input_ids = tf.placeholder(tf.int32, [None, max_seq_length], name='input_ids')
    input_mask = tf.placeholder(tf.int32, [None, max_seq_length], name='input_mask')
    segment_ids = tf.placeholder(tf.int32, [None, max_seq_length], name='segment_ids')
    input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
        'label_ids': label_ids,
        'input_ids': input_ids,
        'input_mask': input_mask,
        'segment_ids': segment_ids,
    })()
    return input_fn
  
export_dir_base = 'gdrive/My Drive/kaggle/jigsaw/workdir/modelexport'
estimator._export_to_tpu = False 
estimator.export_saved_model(export_dir_base=export_dir_base, serving_input_receiver_fn=serving_input_fn)

INFO:tensorflow:Calling model_fn.


I0609 04:45:47.808901 140435720193920 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Running infer on CPU


I0609 04:45:47.811197 140435720193920 tpu_estimator.py:2532] Running infer on CPU


INFO:tensorflow:*** Features ***


...

INFO:tensorflow:Assets written to: gdrive/My Drive/kaggle/jigsaw/workdir/modelexport/temp-b'1560055547'/assets


I0609 04:46:13.389056 140435720193920 builder_impl.py:763] Assets written to: gdrive/My Drive/kaggle/jigsaw/workdir/modelexport/temp-b'1560055547'/assets


INFO:tensorflow:SavedModel written to: gdrive/My Drive/kaggle/jigsaw/workdir/modelexport/temp-b'1560055547'/saved_model.pb


I0609 04:46:19.017761 140435720193920 builder_impl.py:414] SavedModel written to: gdrive/My Drive/kaggle/jigsaw/workdir/modelexport/temp-b'1560055547'/saved_model.pb


b'gdrive/My Drive/kaggle/jigsaw/workdir/modelexport/1560055547'

In [0]:
# This function is not used by this file but is still used by the Colab and
# people who depend on it. 
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features

In [0]:
# This function is not used by this file but is still used by the Colab and
# people who depend on it.
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples,len(LABEL_COLUMNS)], dtype=tf.float32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [28]:
import datetime

# Eval the model.
#eval_examples = processor.get_dev_examples(data_dir)

eval_examples = create_examples(read_tsv(os.path.join(data_dir, "dev.tsv")),"dev")
print(len(eval_examples))
eval_features = convert_examples_to_features(
    eval_examples, label_list, max_seq_length, tokenizer)
print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(eval_examples)))
print('  Batch size = {}'.format(eval_batch_size))

# Eval will be slightly WRONG on the TPU because it will truncate
# the last batch.
eval_steps = int(len(eval_examples) / eval_batch_size)
eval_input_fn = input_fn_builder(
    features=eval_features,
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=True)
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
  print("***** Eval results *****")
  for key in sorted(result.keys()):
    print('  {} = {}'.format(key, str(result[key])))
    writer.write("%s = %s\n" % (key, str(result[key])))


100000
INFO:tensorflow:Writing example 0 of 100000


I0609 04:46:37.301577 140435720193920 <ipython-input-26-7204535d5653>:8] Writing example 0 of 100000


INFO:tensorflow:*** Example ***


I0609 04:46:37.313081 140435720193920 <ipython-input-20-d5b71b59cc05>:91] *** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 106554: gs://jigsawbert/bert-tfhub/models/CoLA_weights/model.ckpt-106554


I0609 04:52:33.294188 140435720193920 estimator.py:2039] Saving 'checkpoint_path' summary for global step 106554: gs://jigsawbert/bert-tfhub/models/CoLA_weights/model.ckpt-106554


INFO:tensorflow:evaluation_loop marked as finished


I0609 04:52:38.688285 140435720193920 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-06-09 04:52:38.695235 *****
***** Eval results *****
  eval_AUC = 0.9452655
  eval_loss = 0.32468823
  global_step = 106554
  loss = 0.35481817


(2) 
with 2 epochs
eval_AUC = 0.9443847
  eval_loss = 0.25044802
  global_step = 106554
  loss = 0.2672604
  

(1) eval_AUC = 0.95403856
  eval_loss = 0.23364438
  global_step = 53277
  loss = 0.24890776
  
  ------

(4)  warmup proportion = 0.05
eval_AUC = 0.9412091
  eval_loss = 0.18474945
  global_step = 53277
  loss = 0.20086

(3) model trained with max sequence  = 222
   eval_AUC = 0.9400057
  eval_loss = 0.18598238
  global_step = 53277
  loss = 0.20572743
  

(2) Model trained on full dataset (100k validation)
***** Eval results *****
  eval_AUC = 0.9129087
  eval_loss = 0.21233697
  global_step = 53277
  loss = 0.2318359


(1)  Model trained on 0.33 data  (100k validation)

eval_AUC = 0.9169841
  eval_loss = 0.20674154
  global_step = 15487
  loss = 0.20077576

-----

In [0]:
# eval_examples = create_examples(read_tsv(os.path.join(data_dir, "dev.tsv")),"dev")
# eval_features = convert_examples_to_features(
#       eval_examples, label_list, max_seq_length, tokenizer)

# eval_steps = int(len(eval_examples) / eval_batch_size)

# eval_input_fn = input_fn_builder(
#       features=eval_features,
#       seq_length=max_seq_length,
#       is_training=False,
#       drop_remainder=False)

#result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
predictions = estimator.predict(eval_input_fn)


In [30]:
len(eval_examples), eval_batch_size, eval_steps

(100000, 32, 3125)

In [31]:

eval_preds = []

#tmp = 0
for example, prediction in zip(eval_examples, predictions):
  #print(prediction)
  eval_preds.append( prediction['probabilities'])
  #tmp += 1
  #if tmp > 5:
  #  break
    
    
#eval_preds

INFO:tensorflow:Calling model_fn.


I0609 04:53:23.544576 140435720193920 estimator.py:1111] Calling model_fn.


INFO:tensorflow:*** Features ***


I0609 04:55:07.969257 140435720193920 <ipython-input-12-32db2d74b3f7>:60] *** Features ***


INFO:tensorflow:  name = input_ids, shape = (4, 222)


...

I0609 04:56:43.190526 140435720193920 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.


I0609 04:56:43.208461 140435720193920 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0609 04:56:43.211824 140435720193920 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


In [32]:
np.vstack(eval_preds)[:,0].shape

(100000,)

In [0]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

# Background Positive, Subgroup Negative
# Here, we restrict the test set to the non-toxic examples that mention the identity and the toxic examples that do not.
def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
#     if (subgroup == 'jewish'):
#         print(subgroup_negative_examples.index)
#         print(non_subgroup_positive_examples.index)        
#         print(len(examples))
    return compute_auc(examples[label]>0.5, examples[model_name])

# Background Negative, Subgroup Positive
# Here, we restrict the test set to the toxic examples that mention the identity and the non-toxic examples that do not. 
def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [34]:
from sklearn import metrics
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

MODEL_NAME = 'model1'

devcopy[MODEL_NAME] = np.vstack(eval_preds)[:,0]

TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(devcopy, identity_columns, MODEL_NAME, 'target')
print(get_final_metric(bias_metrics_df, calculate_overall_auc(devcopy, MODEL_NAME)))
bias_metrics_df

#0.9355333626624606 

#0.9357582515650622 => with 2 epochs

#0.9340219578578859 => with batch size 256 and lr = 4e-5

#0.9334263711867721 => batch size 256 and lr = 6e-5 epoch =2

#0.9352646465071597 => batch size 32 lr = 2e-5 epoch = 1

0.9376376061029494


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.974198,0.862067,black,0.865930,759
2,0.977342,0.848133,homosexual_gay_or_lesbian,0.868232,735
7,0.975084,0.866330,white,0.876013,1389
5,0.971845,0.904793,muslim,0.907487,814
4,0.962046,0.940775,jewish,0.923729,277
0,0.969580,0.935129,male,0.930687,2560
1,0.968418,0.940237,female,0.933840,3501
8,0.976643,0.921723,psychiatric_or_mental_illness,0.935962,315
3,0.966564,0.947323,christian,0.938840,1896


In [47]:
devcopy.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,dummy_1,dummy_2,weights,model1
1704874,6212307,0,Gadabout still has a hangover from all his cel...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-24 15:11:22.943455+00,54,6211410.0,392625,rejected,0,0,0,0,0,0.0,0,4,meh,*,1.0,0.000023
1704875,6212308,0,These bond measures have done exactly what the...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-24 15:11:09.535920+00,13,0.0,392666,approved,0,0,0,0,0,0.0,0,4,meh,*,1.0,0.000017
1704876,6212309,0,“I am glad my legal counsel has been able to f...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-24 15:11:15.866012+00,102,0.0,392649,approved,1,0,0,8,3,0.0,0,4,meh,*,1.0,0.001013
1704877,6212310,0,A bad QB is probably a better evaluator of a b...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-24 15:11:23.221747+00,102,6210462.0,392551,approved,0,0,0,0,0,0.0,0,4,meh,*,1.0,0.000068
1704878,6212311,0,How doesn't the math add up?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-10-24 15:11:59.292134+00,102,6211294.0,392405,approved,0,0,0,0,0,0.0,0,4,meh,*,1.0,0.000061


In [32]:
predict_examples = processor.get_test_examples(data_dir)
num_actual_predict_examples = len(predict_examples)
predict_examples = predict_examples + predict_examples[0:24]


i_features = convert_examples_to_features(predict_examples, label_list, max_seq_length, tokenizer)
predict_input_fn = input_fn_builder(i_features,max_seq_length, is_training=False, drop_remainder=True)
predictions = estimator.predict(predict_input_fn)

tmp = 0
for example, prediction in zip(predict_examples, predictions):
  print(prediction)
  tmp += 1
  if tmp > 5:
    break

INFO:tensorflow:Writing example 0 of 97344


I0608 02:44:16.893095 139780462081920 <ipython-input-21-7204535d5653>:8] Writing example 0 of 97344


INFO:tensorflow:*** Example ***


I0608 02:44:16.899264 139780462081920 <ipython-input-16-d5b71b59cc05>:91] *** Example ***


INFO:tensorflow:guid: test-1


...

INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.


I0608 02:48:34.822487 139780462081920 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0608 02:48:34.825387 139780462081920 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


{'probabilities': array([3.0070543e-05, 3.8412482e-01, 7.8341365e-04, 1.3745427e-03,
       2.1511912e-03, 1.6089171e-02, 1.1850297e-03], dtype=float32)}
{'probabilities': array([1.5109777e-05, 4.4085875e-01, 6.5574050e-04, 1.1936724e-03,
       2.1236241e-03, 3.5345852e-03, 1.3101101e-03], dtype=float32)}
{'probabilities': array([2.4151802e-04, 4.2560691e-01, 2.2999048e-03, 2.4203658e-03,
       2.2106588e-02, 4.3219537e-02, 7.3177516e-03], dtype=float32)}
{'probabilities': array([7.5072050e-05, 3.9046741e-01, 1.1206865e-03, 2.7842820e-03,
       2.4746656e-03, 2.1654248e-02, 2.2300184e-03], dtype=float32)}
{'probabilities': array([0.99902236, 0.49458638, 0.02855045, 0.08963153, 0.02526683,
       0.74889874, 0.01023111], dtype=float32)}
{'probabilities': array([1.9967556e-05, 4.3920496e-01, 6.7454576e-04, 1.2835860e-03,
       1.5472472e-03, 3.1627119e-03, 1.1986494e-03], dtype=float32)}
